In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *

import joblib
import tensorflow as tf
from sklearn.preprocessing import normalize

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [4]:
def create_window(dataset, win_size, start_index=0, end_index=None):
    data = []

    start_index = start_index + win_size
    if end_index is None:
        end_index = len(dataset)

    for i in range(start_index, end_index):
        indices = range(i-win_size, i)
        data.append(dataset[scanners].iloc[indices].values)
    return np.array(data)

In [5]:
def impute(data):
    data.update(min_max_scaling(data))
    data.update(data.rolling('5s').mean())
    data.ffill(inplace=True)

def reg_impute(data):
    for s in scanners:
        dx = coor[0] - ble[s]['X']
        dy = coor[1] - ble[s]['Y']
        d = np.sqrt(np.square(dy) + np.square(dx))
        reg = joblib.load('../Models/linear_distance_models_2/{}.joblib'.format(s))
        ss = pd.Series(reg.predict(d.reshape((-1, 1)))).repeat(data.shape[0])
        ss.index = data.index
        data[s].fillna(ss, inplace=True)

In [6]:
ble = read_ble('../Data/ble.csv')

In [7]:
coor = np.array([22, 6])

In [8]:
model = tf.keras.models.load_model("../Models/LSTM_W10.h5")

In [9]:
client = InfluxDBClient()

In [10]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [11]:
data = format_data(data)

In [12]:
impute(data)
reg_impute(data)
data[scanners] = normalize(data[scanners] + 100)

In [13]:
data

,C400A2E19293,CD4533FFC0E1,D2B6503554D7,DB8B36A69C56,DD697EA75B68,DF231643E227,E13B805C6CB0,E43355CA8B96,E6D9D20DD197,E8FD0B453DC4,E96AF2C858BA,EC72840D9AD3,F1307ECB3B90,F1EDAF28E08A,F69A86823B96,FB2EE01C18CE,FDAE5980F28C
2020-01-16 07:55:04.610000+00:00,0.135911,0.229946,0.167821,0.169149,0.188188,0.215632,0.188423,0.123718,0.158271,0.077197,0.162379,0.221915,0.220643,0.081756,0.740687,0.107231,0.130579
2020-01-16 07:55:04.675000+00:00,0.135923,0.229967,0.167836,0.169164,0.188205,0.215651,0.188440,0.123729,0.158285,0.077204,0.162394,0.221935,0.220663,0.081764,0.740632,0.107241,0.130591
2020-01-16 07:55:04.758000+00:00,0.110288,0.186594,0.600149,0.137259,0.152709,0.174979,0.152900,0.100394,0.128433,0.062643,0.131766,0.180078,0.179046,0.066343,0.600947,0.087015,0.105961
2020-01-16 07:55:04.882000+00:00,0.095332,0.161292,0.518768,0.118647,0.132002,0.151251,0.519888,0.086780,0.111017,0.054149,0.113898,0.155659,0.154767,0.057347,0.519457,0.075216,0.091593
2020-01-16 07:55:04.914000+00:00,0.077805,0.424020,0.423387,0.096832,0.107732,0.123442,0.424302,0.070825,0.090605,0.044193,0.424372,0.127039,0.126311,0.046803,0.423950,0.061387,0.074753
2020-01-16 07:55:04.954000+00:00,0.077805,0.424020,0.423387,0.096832,0.107732,0.123442,0.424302,0.070825,0.090605,0.044193,0.424372,0.127039,0.126311,0.046803,0.423950,0.061387,0.074753
2020-01-16 07:55:05.001000+00:00,0.077805,0.424023,0.423390,0.096833,0.107733,0.123443,0.424305,0.070825,0.090606,0.044193,0.424375,0.127040,0.126312,0.046803,0.423936,0.061387,0.074753
2020-01-16 07:55:05.014000+00:00,0.072007,0.392424,0.391838,0.089617,0.391707,0.114244,0.392684,0.065547,0.083854,0.040900,0.392749,0.117573,0.116899,0.043315,0.392342,0.056812,0.069182
2020-01-16 07:55:05.175000+00:00,0.072007,0.392424,0.391838,0.089617,0.391707,0.114244,0.392684,0.065547,0.083854,0.040900,0.392749,0.117573,0.116899,0.043315,0.392342,0.056812,0.069182
2020-01-16 07:55:05.225000+00:00,0.067428,0.367469,0.366920,0.083918,0.366798,0.106979,0.367712,0.061379,0.078521,0.038299,0.367773,0.110096,0.367591,0.040561,0.367392,0.053199,0.064783


In [14]:
data = create_window(data, 10)

In [15]:
coor = model.predict(data).mean(axis=0)

In [16]:
np.expand_dims(coor, axis=0)

array([[[20.42818  ,  6.498864 ],
        [20.501484 ,  6.8000455],
        [20.491545 ,  6.9454284],
        [20.509157 ,  6.9952226],
        [20.527721 ,  6.9862223],
        [20.550566 ,  6.9649806],
        [20.59375  ,  6.9406753],
        [20.654892 ,  6.9175673],
        [20.72964  ,  6.896083 ],
        [20.822407 ,  6.8789   ]]], dtype=float32)